In [1]:
# Para llevar el registro
!pip install -q comet_ml
import comet_ml
comet_ml.init(project_name = 'Regularizer')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.0.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.0 requires jupyterlab<5.0.0a0,>=4.0.6, but you have jupyterlab 4.0.5 which is incompatible.
Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)


Comet API key:  ·························


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [2]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, Conv2D, MaxPooling2D, Concatenate, PReLU
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Adadelta
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.regularizers import l2

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Se crea el experimento en Comet
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging = True,
    auto_histogram_gradient_logging = True,
    auto_histogram_activation_logging = True,
    log_code = True
)
experiment.set_name('Try_6 [from: FunctionalAPI]')

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/gurumasterahb/regularizer/43be4131cd8641718803b2160871e5cb



In [4]:
# Cargar los datos de los txt
file_attr = os.path.join('/kaggle/input/img-align-celeba-partition/attr.csv')
attr = pd.read_csv(file_attr)

# Imágenes
i_test = '/kaggle/input/img-align-celeba-partition/align_partition/align_test'
i_train = '/kaggle/input/img-align-celeba-partition/align_partition/align_train'
i_val = '/kaggle/input/img-align-celeba-partition/align_partition/align_val'

In [5]:
# Parámetros
i_width = int(178 * 1.2)
i_height = int(218 * 1.2)
input_shape = (i_width, i_height, 3)

parameters = {
    'num_class' : 38,
    'epochs' : 60,
    'batch_size' : 64,
    'loss' : 'binary_crossentropy',
    'optimizer' : 'RMSprop',
    'num_train' : 162770,
    'num_val' : 19867,
    'num_test' : 19962,
}
experiment.log_parameters(parameters)

In [6]:
# Solo toma el entero
epoch_steps = parameters['num_train'] // (parameters['batch_size'] * 6) # Training
val_steps = parameters['num_val'] // (parameters['batch_size'] * 6) # Validation
test_steps = parameters['num_test'] // (parameters['batch_size'] * 6) # Testing

In [7]:
gen_train = ImageDataGenerator(rescale = 1./255.)
train = gen_train.flow_from_dataframe(
    attr,
    directory = i_train,
    x_col = attr.columns[0],  # Nombre de la columna que contiene los nombres de las imágenes
    y_col = attr.columns[1:],  # Lista de nombres de las columnas de atributos
    target_size = (i_width, i_height),
    batch_size = parameters['batch_size'],
    class_mode = 'raw',
    shuffle = True
)

gen_val = ImageDataGenerator(rescale = 1./255.)
val = gen_val.flow_from_dataframe(
    attr,
    directory = i_val,
    x_col = attr.columns[0],  # Nombre de la columna que contiene los nombres de las imágenes
    y_col = attr.columns[1:],  # Lista de nombres de las columnas de atributos
    target_size = (i_width, i_height),
    batch_size = parameters['batch_size'],
    class_mode = 'raw',
    shuffle = True
)

Found 162770 validated image filenames.


/opt/conda/lib/python3.10/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 39829 invalid image filename(s) in x_col="202599". These filename(s) will be ignored.
  warnings.warn(


Found 19867 validated image filenames.


/opt/conda/lib/python3.10/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 182732 invalid image filename(s) in x_col="202599". These filename(s) will be ignored.
  warnings.warn(


In [8]:
# # Arquitectura
# # Functional API
# inputs = Input(shape = input_shape)

# conv1 = Conv2D(32, (3,3), activation = 'relu')(inputs)
# mpool1 = MaxPooling2D(pool_size = (2,2))(conv1)

# conv2 = Conv2D(64, (3,3), activation = 'relu', kernel_regularizer = l2(0.0001))(mpool1)
# mpool2 = MaxPooling2D(pool_size = (2,2))(conv2)

# conv3 = Conv2D(128, (3,3), activation = 'relu')(mpool2)
# mpool3 = MaxPooling2D(pool_size = (2,2))(conv3)

# conv4 = Conv2D(256, (3,3), activation = 'relu')(mpool3)
# mpool4 = MaxPooling2D(pool_size = (2,2))(conv4)

# conv5 = Conv2D(512, (3,3), activation = 'relu', kernel_regularizer = l2(0.0001))(mpool4)
# mpool5 = MaxPooling2D(pool_size = (2,2))(conv5)

# conv6 = Conv2D(256, (3,3), activation = 'relu')(mpool5)
# mpool6 = MaxPooling2D(pool_size = (2,2))(conv6)

# x_1 = Flatten()(mpool6)

# x_2 = Dense(1024, activation = 'relu')(x_1)

# x_3 = Dense(1024, activation = 'relu')(x_2)
# x_4 = Dropout(0.15)(x_3)

# x_5 = Dense(512, activation = 'PReLU', kernel_regularizer = l2(0.0001))(x_4)

# x_6 = Dense(256, activation = 'PReLU')(x_5)

# x_7 = Dense(128, activation = 'relu')(x_6)

# x_out = Dense(parameters['num_class'], activation = 'sigmoid')(x_7)

# model = tf.keras.Model(inputs = inputs, outputs = x_out)
# model.summary()

In [15]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = input_shape))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3), activation = 'relu', kernel_regularizer = l2(0.0001)))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(256, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(512, (3,3), activation = 'relu', kernel_regularizer = l2(0.0001)))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(256, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())

model.add(Dense(1024, activation = 'relu'))

model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.15))
          
model.add(Dense(512, activation = 'PReLU', kernel_regularizer = l2(0.0001)))

model.add(Dense(256, activation = 'PReLU'))

model.add(Dense(128, activation = 'relu'))

model.add(Dense(parameters['num_class'], activation = 'sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 211, 259, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 105, 129, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 103, 127, 64)      18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 51, 63, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 49, 61, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 24, 30, 128)      

In [16]:
# Checkpoint para el mejor modelo (val_accuracy)
filepath = 'best_val_loss.h5'
checkpoint = ModelCheckpoint(
    filepath,
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    mode = 'min'
)

In [17]:
# Compilación y entrenamiento
model.compile(
    loss = parameters['loss'],
    optimizer = RMSprop(learning_rate = 0.001),
    metrics = ['binary_accuracy']
)

model.fit(
    train,
    batch_size = parameters['batch_size'],
    epochs = parameters['epochs'],
    verbose = 1,
    validation_data = val,
    steps_per_epoch = epoch_steps,
    validation_steps = val_steps,
    callbacks = [checkpoint]
)

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE
COMET WARNING: auto_histogram_gradient_logging is True, but inputs and targets are not available; unable to log gradients
COMET WARNING: auto_histogram_activation_logging is True, but inputs are not available; unable to log activations


Epoch 1/60
423/423 [==============================] - ETA: 0s - loss: 0.5220 - binary_accuracy: 0.8090
Epoch 1: val_loss improved from inf to 0.45545, saving model to best_val_loss.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


423/423 [==============================] - 281s 634ms/step - loss: 0.5220 - binary_accuracy: 0.8090 - val_loss: 0.4555 - val_binary_accuracy: 0.8274
Epoch 2/60
423/423 [==============================] - ETA: 0s - loss: 0.4099 - binary_accuracy: 0.8428
Epoch 2: val_loss improved from 0.45545 to 0.36468, saving model to best_val_loss.h5
423/423 [==============================] - 261s 618ms/step - loss: 0.4099 - binary_accuracy: 0.8428 - val_loss: 0.3647 - val_binary_accuracy: 0.8609
Epoch 3/60
423/423 [==============================] - ETA: 0s - loss: 0.3461 - binary_accuracy: 0.8646
Epoch 3: val_loss improved from 0.36468 to 0.32714, saving model to best_val_loss.h5
423/423 [==============================] - 221s 522ms/step - loss: 0.3461 - binary_accuracy: 0.8646 - val_loss: 0.3271 - val_binary_accuracy: 0.8714
Epoch 4/60
423/423 [==============================] - ETA: 0s - loss: 0.3159 - binary_accuracy: 0.8723
Epoch 4: val_loss improved from 0.32714 to 0.29872, saving model to best_v

In [18]:
experiment.log_model("Facial_r", filepath)
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/regularizer/43be4131cd8641718803b2160871e5cb
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     batch_binary_accuracy [2580]         : (0.4609375, 0.9268091917037964)
COMET INFO:     batch_loss [2580]                    : (0.19610679149627686, 0.8063083291053772)
COMET INFO:     binary_accuracy [60]                 : (0.8090057969093323, 0.9148674011230469)
COMET INFO:     epoch_duration [60]                  : (91.45809784200083, 281.05137145599997)
COMET INFO:     loss [60]                            : (0.21310879290103912, 0.522043764591217)
COMET INFO:     val_binary_accuracy [60]             : (0.827439